In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import auth


In [2]:
JSON_PATH = '../hanabi-ee185-firebase-adminsdk-3ezuk-ad35ec06cd.key'
cred = credentials.Certificate(JSON_PATH)
firebase_admin.initialize_app(cred)

In [5]:
app = firebase_admin.get_app()
print(app.project_id)

hanabi-ee185


In [6]:
client = auth.Client(app)

In [16]:
phone = "+818053156298"

In [23]:
user = auth.get_user_by_phone_number(phone)
print('Successfully fetched user data: {0}'.format(user.uid))
target_uid = user.uid

Successfully fetched user data: i2guWmQOKuUZAvVapPeiGMvyH1T2


In [ ]:
auth.create_user(phone_number="+818053156298")

In [24]:
auth.delete_user(target_uid)

In [2]:
import pymysql.cursors
import requests
import pandas as pd
import numpy as np
from pytz import timezone
import datetime
import os
import json


In [5]:
def connect_to_mysql():
    connection = pymysql.connect(host="hanabi.cjmk7x3nn2rr.ap-northeast-1.rds.amazonaws.com", # プロキシに変更する,
                                 user="admin",
                                 password="hanabi20230612",
                                 database="hanabi",
                                 cursorclass=pymysql.cursors.DictCursor)
    return connection


In [6]:
def get_receipts():
    connection = connect_to_mysql()
    with connection:
        with connection.cursor() as cursor:
            try:
                return_list = []
                sql = "SELECT * FROM apple_receipts;"
                cursor.execute(sql)
                results = cursor.fetchall()
                for raw in results:
                    return_list.append(raw)
            except Exception as e:
                print("get campaign error :",e)
    return return_list

In [7]:
receipts = get_receipts()

In [8]:
data = receipts[0]["receipt_data"]

In [9]:
data

'TUlJbVZnWUpLb1pJaHZjTkFRY0NvSUltUnpDQ0prTUNBUUV4Q3pBSkJnVXJEZ01DR2dVQU1JSVZsQVlKS29aSWh2Y05BUWNCb0lJVmhRU0NGWUV4Z2hWOU1Bb0NBUWdDQVFFRUFoWUFNQW9DQVJRQ0FRRUVBZ3dBTUFzQ0FRRUNBUUVFQXdJQkFEQUxBZ0VMQWdFQkJBTUNBUUF3Q3dJQkR3SUJBUVFEQWdFQU1Bc0NBUkFDQVFFRUF3SUJBREFMQWdFWkFnRUJCQU1DQVFNd0RBSUJBd0lCQVFRRURBSXhPREFNQWdFS0FnRUJCQVFXQWpRck1Bd0NBUTRDQVFFRUJBSUNBSjB3RFFJQkRRSUJBUVFGQWdNQ2Mxa3dEUUlCRXdJQkFRUUZEQU14TGpBd0RnSUJDUUlCQVFRR0FnUlFNekF4TUJnQ0FRUUNBUUlFRUJrcmtnOHprckFwRXFmVzFCMjl4K2d3R3dJQkFBSUJBUVFUREJGUWNtOWtkV04wYVc5dVUyRnVaR0p2ZURBY0FnRUZBZ0VCQkJRSVJwc01ENUVCNmJBQUNiOUx2V0JsRFB0K3BEQWVBZ0VDQWdFQkJCWU1GR052YlM1b1lXNWhZbWxwYm1NdWFHRnVZV0pwTUI0Q0FRd0NBUUVFRmhZVU1qQXlNeTB3T1MweE0xUXhOVG8wTURvMU9Gb3dIZ0lCRWdJQkFRUVdGaFF5TURFekxUQTRMVEF4VkRBM09qQXdPakF3V2pCUEFnRUdBZ0VCQkVlOVpKbFBMVXN6dU5UZGhiTC9SY0FIRFlNYW44Y3VaRHIyeThlWkpLZXdLVFdrdEpoKzdMNCtGaGtiMDBYbUVWTDZuK1N6YnVzTE5aRFNvRFVyNmZobXFqc21XeG1wK0RCUUFnRUhBZ0VCQkVoV3FYRUlyZEJtTVJ4ZEg3eXJSd2dCQkZwVGtnbkpCTXNQVkpERklBb3pNbVdKR1dZSm1aQ2g1QU9kQWQzT2k

In [10]:
import base64
import asn1
import requests

In [11]:
textBytes = base64.b64decode(data.encode())
# textBytes = base64.b64decode(data)

In [12]:
receipt = textBytes.decode()
print(receipt)

MIImVgYJKoZIhvcNAQcCoIImRzCCJkMCAQExCzAJBgUrDgMCGgUAMIIVlAYJKoZIhvcNAQcBoIIVhQSCFYExghV9MAoCAQgCAQEEAhYAMAoCARQCAQEEAgwAMAsCAQECAQEEAwIBADALAgELAgEBBAMCAQAwCwIBDwIBAQQDAgEAMAsCARACAQEEAwIBADALAgEZAgEBBAMCAQMwDAIBAwIBAQQEDAIxODAMAgEKAgEBBAQWAjQrMAwCAQ4CAQEEBAICAJ0wDQIBDQIBAQQFAgMCc1kwDQIBEwIBAQQFDAMxLjAwDgIBCQIBAQQGAgRQMzAxMBgCAQQCAQIEEBkrkg8zkrApEqfW1B29x+gwGwIBAAIBAQQTDBFQcm9kdWN0aW9uU2FuZGJveDAcAgEFAgEBBBQIRpsMD5EB6bAACb9LvWBlDPt+pDAeAgECAgEBBBYMFGNvbS5oYW5hYmlpbmMuaGFuYWJpMB4CAQwCAQEEFhYUMjAyMy0wOS0xM1QxNTo0MDo1OFowHgIBEgIBAQQWFhQyMDEzLTA4LTAxVDA3OjAwOjAwWjBPAgEGAgEBBEe9ZJlPLUszuNTdhbL/RcAHDYMan8cuZDr2y8eZJKewKTWktJh+7L4+Fhkb00XmEVL6n+SzbusLNZDSoDUr6fhmqjsmWxmp+DBQAgEHAgEBBEhWqXEIrdBmMRxdH7yrRwgBBFpTkgnJBMsPVJDFIAozMmWJGWYJmZCh5AOdAd3Oi/gty+IQ1YGlEIGJ12HvaEfKF9McuNdQuQcwggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMC

In [20]:
receipt = "MIJNLgYJKoZIhvcNAQcCoIJNHzCCTRsCAQExCzAJBgUrDgMCGgUAMII8bAYJKoZIhvcNAQcBoII8XQSCPFkxgjxVMAoCAQgCAQEEAhYAMAoCARQCAQEEAgwAMAsCAQECAQEEAwIBADALAgEDAgEBBAMMATYwCwIBCwIBAQQDAgEAMAsCAQ8CAQEEAwIBADALAgEQAgEBBAMCAQAwCwIBGQIBAQQDAgEDMAwCAQoCAQEEBBYCNCswDAIBDgIBAQQEAgIAnTANAgENAgEBBAUCAwJzWTANAgETAgEBBAUMAzEuMDAOAgEJAgEBBAYCBFAzMDIwGAIBBAIBAgQQSn0a6EbUxlak1UwWIcBgXDAbAgEAAgEBBBMMEVByb2R1Y3Rpb25TYW5kYm94MBwCAQUCAQEEFOPunfa5VSZprVV278f8KNxqXM4wMB4CAQICAQEEFgwUY29tLmhhbmFiaWluYy5oYW5hYmkwHgIBDAIBAQQWFhQyMDIzLTEwLTEwVDE0OjUwOjMzWjAeAgESAgEBBBYWFDIwMTMtMDgtMDFUMDc6MDA6MDBaMEgCAQcCAQEEQLNXYQ0A/9QwDtLaofG4hyw+Zx4BaLMq72sPpsW56nx49n4uDzpD/BudJS52QzVZwq8yrOCw01NaI7coSoE3h8swVAIBBgIBAQRMVz7qvFMpbFwT3Lr8HKzsoR90y2e+kB2UqznxpYuyZ32/kxUmPIzSPDChNes8SOO26RyXCbGCSf3QZv/1cQeH72ccTAyqt/AJbSVyQDCCAZgCARECAQEEggGOMYIBijALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S8hAsjAbAgIGpwIBAQQSDBAyMDAwMDAwNDE4MzA2NDQxMBsCAgapAgEBBBIMEDIwMDAwMDA0MTgzMDY0NDEwHwICBqgCAQEEFhYUMjAyMy0wOS0yMVQwMTo0MTo0NVowHwICBqoCAQEEFhYUMjAyMy0wOS0yMVQwMTo0MTo1MlowHwICBqwCAQEEFhYUMjAyMy0wOS0yMVQwMTo0Njo0NVowKAICBqYCAQEEHwwdY29tX2hhbmFiaS52aXAub25lX21vbnRoX3BsYW4wggGYAgERAgEBBIIBjjGCAYowCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/UvIQLMwGwICBqcCAQEEEgwQMjAwMDAwMDQxODMwNzg3NjAbAgIGqQIBAQQSDBAyMDAwMDAwNDE4MzA2NDQxMB8CAgaoAgEBBBYWFDIwMjMtMDktMjFUMDE6NDY6NDVaMB8CAgaqAgEBBBYWFDIwMjMtMDktMjFUMDE6NDE6NTJaMB8CAgasAgEBBBYWFDIwMjMtMDktMjFUMDE6NTE6NDVaMCgCAgamAgEBBB8MHWNvbV9oYW5hYmkudmlwLm9uZV9tb250aF9wbGFuMIIBmAIBEQIBAQSCAY4xggGKMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LyEGAMBsCAganAgEBBBIMEDIwMDAwMDA0MTgzMTEwNjcwGwICBqkCAQEEEgwQMjAwMDAwMDQxODMwNjQ0MTAfAgIGqAIBAQQWFhQyMDIzLTA5LTIxVDAxOjUxOjQ1WjAfAgIGqgIBAQQWFhQyMDIzLTA5LTIxVDAxOjQxOjUyWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTIxVDAxOjU2OjQ1WjAoAgIGpgIBAQQfDB1jb21faGFuYWJpLnZpcC5vbmVfbW9udGhfcGxhbjCCAZgCARECAQEEggGOMYIBijALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S8hCnjAbAgIGpwIBAQQSDBAyMDAwMDAwNDE4MzEyNzg3MBsCAgapAgEBBBIMEDIwMDAwMDA0MTgzMDY0NDEwHwICBqgCAQEEFhYUMjAyMy0wOS0yMVQwMTo1Njo0NVowHwICBqoCAQEEFhYUMjAyMy0wOS0yMVQwMTo0MTo1MlowHwICBqwCAQEEFhYUMjAyMy0wOS0yMVQwMjowMTo0NVowKAICBqYCAQEEHwwdY29tX2hhbmFiaS52aXAub25lX21vbnRoX3BsYW4wggGYAgERAgEBBIIBjjGCAYowCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/UvIQ8UwGwICBqcCAQEEEgwQMjAwMDAwMDQxODMxNTEzNjAbAgIGqQIBAQQSDBAyMDAwMDAwNDE4MzA2NDQxMB8CAgaoAgEBBBYWFDIwMjMtMDktMjFUMDI6MDE6NDVaMB8CAgaqAgEBBBYWFDIwMjMtMDktMjFUMDE6NDE6NTJaMB8CAgasAgEBBBYWFDIwMjMtMDktMjFUMDI6MDY6NDVaMCgCAgamAgEBBB8MHWNvbV9oYW5hYmkudmlwLm9uZV9tb250aF9wbGFuMIIBmAIBEQIBAQSCAY4xggGKMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LyET+MBsCAganAgEBBBIMEDIwMDAwMDA0MTgzMTgxMDcwGwICBqkCAQEEEgwQMjAwMDAwMDQxODMwNjQ0MTAfAgIGqAIBAQQWFhQyMDIzLTA5LTIxVDAyOjA2OjQ1WjAfAgIGqgIBAQQWFhQyMDIzLTA5LTIxVDAxOjQxOjUyWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTIxVDAyOjExOjQ1WjAoAgIGpgIBAQQfDB1jb21faGFuYWJpLnZpcC5vbmVfbW9udGhfcGxhbjCCAZgCARECAQEEggGOMYIBijALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S8hGODAbAgIGpwIBAQQSDBAyMDAwMDAwNDE4MzIyMDg2MBsCAgapAgEBBBIMEDIwMDAwMDA0MTgzMDY0NDEwHwICBqgCAQEEFhYUMjAyMy0wOS0yMVQwMjoxMTo0NVowHwICBqoCAQEEFhYUMjAyMy0wOS0yMVQwMTo0MTo1MlowHwICBqwCAQEEFhYUMjAyMy0wOS0yMVQwMjoxNjo0NVowKAICBqYCAQEEHwwdY29tX2hhbmFiaS52aXAub25lX21vbnRoX3BsYW4wggGYAgERAgEBBIIBjjGCAYowCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/UvIR6YwGwICBqcCAQEEEgwQMjAwMDAwMDQxODMyNDQyMTAbAgIGqQIBAQQSDBAyMDAwMDAwNDE4MzA2NDQxMB8CAgaoAgEBBBYWFDIwMjMtMDktMjFUMDI6MTY6NDVaMB8CAgaqAgEBBBYWFDIwMjMtMDktMjFUMDE6NDE6NTJaMB8CAgasAgEBBBYWFDIwMjMtMDktMjFUMDI6MjE6NDVaMCgCAgamAgEBBB8MHWNvbV9oYW5hYmkudmlwLm9uZV9tb250aF9wbGFuMIIBmAIBEQIBAQSCAY4xggGKMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LyEj5MBsCAganAgEBBBIMEDIwMDAwMDA0MTgzMjU3MzQwGwICBqkCAQEEEgwQMjAwMDAwMDQxODMwNjQ0MTAfAgIGqAIBAQQWFhQyMDIzLTA5LTIxVDAyOjIxOjQ1WjAfAgIGqgIBAQQWFhQyMDIzLTA5LTIxVDAxOjQxOjUyWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTIxVDAyOjI2OjQ1WjAoAgIGpgIBAQQfDB1jb21faGFuYWJpLnZpcC5vbmVfbW9udGhfcGxhbjCCAZgCARECAQEEggGOMYIBijALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S8hKTTAbAgIGpwIBAQQSDBAyMDAwMDAwNDE4MzMxMzExMBsCAgapAgEBBBIMEDIwMDAwMDA0MTgzMDY0NDEwHwICBqgCAQEEFhYUMjAyMy0wOS0yMVQwMjoyNjo0NVowHwICBqoCAQEEFhYUMjAyMy0wOS0yMVQwMTo0MTo1MlowHwICBqwCAQEEFhYUMjAyMy0wOS0yMVQwMjozMTo0NVowKAICBqYCAQEEHwwdY29tX2hhbmFiaS52aXAub25lX21vbnRoX3BsYW4wggGYAgERAgEBBIIBjjGCAYowCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/UvIS48wGwICBqcCAQEEEgwQMjAwMDAwMDQxODMzMzQxODAbAgIGqQIBAQQSDBAyMDAwMDAwNDE4MzA2NDQxMB8CAgaoAgEBBBYWFDIwMjMtMDktMjFUMDI6MzE6NThaMB8CAgaqAgEBBBYWFDIwMjMtMDktMjFUMDE6NDE6NTJaMB8CAgasAgEBBBYWFDIwMjMtMDktMjFUMDI6MzY6NThaMCgCAgamAgEBBB8MHWNvbV9oYW5hYmkudmlwLm9uZV9tb250aF9wbGFuMIIBmAIBEQIBAQSCAY4xggGKMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LyEzrMBsCAganAgEBBBIMEDIwMDAwMDA0MTgzMzQ3MzAwGwICBqkCAQEEEgwQMjAwMDAwMDQxODMwNjQ0MTAfAgIGqAIBAQQWFhQyMDIzLTA5LTIxVDAyOjM2OjU4WjAfAgIGqgIBAQQWFhQyMDIzLTA5LTIxVDAxOjQxOjUyWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTIxVDAyOjQxOjU4WjAoAgIGpgIBAQQfDB1jb21faGFuYWJpLnZpcC5vbmVfbW9udGhfcGxhbjCCAZsCARECAQEEggGRMYIBjTALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S7zlkjAbAgIGpwIBAQQSDBAyMDAwMDAwNDExNTc5NTgyMBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xMlQxNTozMTowM1owHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xMlQxNTozNjowM1owKwICBqYCAQEEIgwgY29tX2hhbmFiaS5ub3JtYWwub25lX21vbnRoX3BsYW4wggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/Uu85ZMwGwICBqcCAQEEEgwQMjAwMDAwMDQxMTU4MjY2NTAbAgIGqQIBAQQSDBAyMDAwMDAwNDExNTc5NTgyMB8CAgaoAgEBBBYWFDIwMjMtMDktMTJUMTU6MzY6MDNaMB8CAgaqAgEBBBYWFDIwMjMtMDktMTJUMTU6MzE6MTBaMB8CAgasAgEBBBYWFDIwMjMtMDktMTJUMTU6NDE6MDNaMCsCAgamAgEBBCIMIGNvbV9oYW5hYmkubm9ybWFsLm9uZV9tb250aF9wbGFuMIIBmwIBEQIBAQSCAZExggGNMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LvObqMBsCAganAgEBBBIMEDIwMDAwMDA0MTE1ODcxMDQwGwICBqkCAQEEEgwQMjAwMDAwMDQxMTU3OTU4MjAfAgIGqAIBAQQWFhQyMDIzLTA5LTEyVDE1OjQxOjAzWjAfAgIGqgIBAQQWFhQyMDIzLTA5LTEyVDE1OjMxOjEwWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTEyVDE1OjQ2OjAzWjArAgIGpgIBAQQiDCBjb21faGFuYWJpLm5vcm1hbC5vbmVfbW9udGhfcGxhbjCCAZsCARECAQEEggGRMYIBjTALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S7zo4DAbAgIGpwIBAQQSDBAyMDAwMDAwNDExNTkxNTYwMBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xMlQxNTo0NjowM1owHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xMlQxNTo1MTowM1owKwICBqYCAQEEIgwgY29tX2hhbmFiaS5ub3JtYWwub25lX21vbnRoX3BsYW4wggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/Uu86m8wGwICBqcCAQEEEgwQMjAwMDAwMDQxMTU5NTQxOTAbAgIGqQIBAQQSDBAyMDAwMDAwNDExNTc5NTgyMB8CAgaoAgEBBBYWFDIwMjMtMDktMTJUMTU6NTE6MDNaMB8CAgaqAgEBBBYWFDIwMjMtMDktMTJUMTU6MzE6MTBaMB8CAgasAgEBBBYWFDIwMjMtMDktMTJUMTU6NTY6MDNaMCsCAgamAgEBBCIMIGNvbV9oYW5hYmkubm9ybWFsLm9uZV9tb250aF9wbGFuMIIBmwIBEQIBAQSCAZExggGNMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LvOwCMBsCAganAgEBBBIMEDIwMDAwMDA0MTE1OTkwNjEwGwICBqkCAQEEEgwQMjAwMDAwMDQxMTU3OTU4MjAfAgIGqAIBAQQWFhQyMDIzLTA5LTEyVDE1OjU2OjAzWjAfAgIGqgIBAQQWFhQyMDIzLTA5LTEyVDE1OjMxOjEwWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTEyVDE2OjAxOjAzWjArAgIGpgIBAQQiDCBjb21faGFuYWJpLm5vcm1hbC5vbmVfbW9udGhfcGxhbjCCAZsCARECAQEEggGRMYIBjTALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S7ztwzAbAgIGpwIBAQQSDBAyMDAwMDAwNDExNjAyOTY0MBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xMlQxNjowMTowM1owHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xMlQxNjowNjowM1owKwICBqYCAQEEIgwgY29tX2hhbmFiaS5ub3JtYWwub25lX21vbnRoX3BsYW4wggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/Uu874EwGwICBqcCAQEEEgwQMjAwMDAwMDQxMTYwNjMyODAbAgIGqQIBAQQSDBAyMDAwMDAwNDExNTc5NTgyMB8CAgaoAgEBBBYWFDIwMjMtMDktMTJUMTY6MDY6MDNaMB8CAgaqAgEBBBYWFDIwMjMtMDktMTJUMTU6MzE6MTBaMB8CAgasAgEBBBYWFDIwMjMtMDktMTJUMTY6MTE6MDNaMCsCAgamAgEBBCIMIGNvbV9oYW5hYmkubm9ybWFsLm9uZV9tb250aF9wbGFuMIIBmwIBEQIBAQSCAZExggGNMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LvPEJMBsCAganAgEBBBIMEDIwMDAwMDA0MTE2MTAxMzYwGwICBqkCAQEEEgwQMjAwMDAwMDQxMTU3OTU4MjAfAgIGqAIBAQQWFhQyMDIzLTA5LTEyVDE2OjEyOjExWjAfAgIGqgIBAQQWFhQyMDIzLTA5LTEyVDE1OjMxOjEwWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTEyVDE2OjE3OjExWjArAgIGpgIBAQQiDCBjb21faGFuYWJpLm5vcm1hbC5vbmVfbW9udGhfcGxhbjCCAZsCARECAQEEggGRMYIBjTALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S7zzGTAbAgIGpwIBAQQSDBAyMDAwMDAwNDExNjEzMDgzMBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xMlQxNjoxNzoxMVowHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xMlQxNjoyMjoxMVowKwICBqYCAQEEIgwgY29tX2hhbmFiaS5ub3JtYWwub25lX21vbnRoX3BsYW4wggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwDAICBrcCAQEEAwIBADAMAgIGugIBAQQDAgEAMBICAgavAgEBBAkCBwca/Uu89IkwGwICBqcCAQEEEgwQMjAwMDAwMDQxMTYxNjE1MjAbAgIGqQIBAQQSDBAyMDAwMDAwNDExNTc5NTgyMB8CAgaoAgEBBBYWFDIwMjMtMDktMTJUMTY6MjI6MTFaMB8CAgaqAgEBBBYWFDIwMjMtMDktMTJUMTU6MzE6MTBaMB8CAgasAgEBBBYWFDIwMjMtMDktMTJUMTY6Mjc6MTFaMCsCAgamAgEBBCIMIGNvbV9oYW5hYmkubm9ybWFsLm9uZV9tb250aF9wbGFuMIIBmwIBEQIBAQSCAZExggGNMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQMwDAICBq4CAQEEAwIBADAMAgIGsQIBAQQDAgEAMAwCAga3AgEBBAMCAQAwDAICBroCAQEEAwIBADASAgIGrwIBAQQJAgcHGv1LvPYFMBsCAganAgEBBBIMEDIwMDAwMDA0MTE2MTc3MzgwGwICBqkCAQEEEgwQMjAwMDAwMDQxMTU3OTU4MjAfAgIGqAIBAQQWFhQyMDIzLTA5LTEyVDE2OjI3OjExWjAfAgIGqgIBAQQWFhQyMDIzLTA5LTEyVDE1OjMxOjEwWjAfAgIGrAIBAQQWFhQyMDIzLTA5LTEyVDE2OjMyOjExWjArAgIGpgIBAQQiDCBjb21faGFuYWJpLm5vcm1hbC5vbmVfbW9udGhfcGxhbjCCAZ0CARECAQEEggGTMYIBjzALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S7z3UjAbAgIGpwIBAQQSDBAyMDAwMDAwNDEyNjI1OTU4MBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xM1QxNTo0MTo1MFowHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xM1QxNTo1Njo1MFowLQICBqYCAQEEJAwiY29tX2hhbmFiaS5ub3JtYWwudGhyZWVfbW9udGhfcGxhbjCCAZ0CARECAQEEggGTMYIBjzALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S76HtDAbAgIGpwIBAQQSDBAyMDAwMDAwNDEyNjM2Mjg1MBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xM1QxNTo1Njo1MFowHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xM1QxNjoxMTo1MFowLQICBqYCAQEEJAwiY29tX2hhbmFiaS5ub3JtYWwudGhyZWVfbW9udGhfcGxhbjCCAZ0CARECAQEEggGTMYIBjzALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga6AgEBBAMCAQAwEgICBq8CAQEECQIHBxr9S76MwDAbAgIGpwIBAQQSDBAyMDAwMDAwNDEyNjQ2MzE3MBsCAgapAgEBBBIMEDIwMDAwMDA0MTE1Nzk1ODIwHwICBqgCAQEEFhYUMjAyMy0wOS0xM1QxNjoxMTo1MFowHwICBqoCAQEEFhYUMjAyMy0wOS0xMlQxNTozMToxMFowHwICBqwCAQEEFhYUMjAyMy0wOS0xM1QxNjoyNjo1MFowLQICBqYCAQEEJAwiY29tX2hhbmFiaS5ub3JtYWwudGhyZWVfbW9udGhfcGxhbjCCAZ0CARECAQEEggGTMYIBjzALAgIGrQIBAQQCDAAwCwICBrACAQEEAhYAMAsCAgayAgEBBAIMADALAgIGswIBAQQCDAAwCwICBrQCAQEEAgwAMAsCAga1AgEBBAIMADALAgIGtgIBAQQCDAAwDAICBqUCAQEEAwIBATAMAgIGqwIBAQQDAgEDMAwCAgauAgEBBAMCAQAwDAICBrECAQEEAwIBADAMAgIGtwIBAQQDAgEAMAwCAga"

In [21]:
url = "https://buy.itunes.apple.com/verifyReceipt"
params = {"receipt-data": receipt,
          "password": "312f4d460b7e456d92b51d51ccb15fe5",
          "exclude-old-transactions": True
          }
response = requests.post(url, json=params)

In [22]:
response = response.json()
print(response)

{'status': 21002}


In [26]:
environment = response["environment"]
transaction_id = response["latest_receipt_info"][0]["transaction_id"]
product_id = response["latest_receipt_info"][0]["product_id"]
original_transaction_id = response["latest_receipt_info"][0]["original_transaction_id"]
purchase_data = response["latest_receipt_info"][0]["purchase_date"]
original_purchase_data = response["latest_receipt_info"][0]["original_purchase_date"]
expires_date = response["latest_receipt_info"][0]["expires_date"]
web_order_line_item_id = response["latest_receipt_info"][0]["web_order_line_item_id"]


Sandbox


In [61]:
sample = "MIITuAYJKoZIhvcNAQcCoIITqTCCE6UCAQExCzAJBgUrDgMCGgUAMIIDWQYJKoZIhvcNAQcBoIIDSgSCA0YxggNCMAoCAQgCAQEEAhYAMAoCARQCAQEEAgwAMAsCAQECAQEEAwIBADALAgEDAgEBBAMMATMwCwIBCwIBAQQDAgEAMAsCAQ4CAQEEAwIBWjALAgEPAgEBBAMCAQAwCwIBEAIBAQQDAgEAMAsCARkCAQEEAwIBAzAMAgEKAgEBBAQWAjQrMA0CAQ0CAQEEBQIDAYfPMA0CARMCAQEEBQwDMS4wMA4CAQkCAQEEBgIEUDI1MDAYAgEEAgECBBA04jSbC9Zi5OwSemv9EK8kMBsCAQACAQEEEwwRUHJvZHVjdGlvblNhbmRib3gwHAIBAgIBAQQUDBJjb20uYmVsaXZlLmFwcC5pb3MwHAIBBQIBAQQUJzhO1BR1kxOVGrCEqQLkwvUuZP8wHgIBDAIBAQQWFhQyMDE4LTExLTEzVDE2OjQ2OjMxWjAeAgESAgEBBBYWFDIwMTMtMDgtMDFUMDc6MDA6MDBaMD0CAQcCAQEENedAPSDSwFz7IoNyAPZTI59czwFA1wkme6h1P/iicVNxpR8niuvFpKYx1pqnKR34cdDeJIzMMFECAQYCAQEESfQpXyBVFno5UWwqDFaMQ/jvbkZCDvz3/6RVKPU80KMCSp4onID0/AWet6BjZgagzrXtsEEdVLzfZ1ocoMuCNTOMyiWYS8uJj0YwggFKAgERAgEBBIIBQDGCATwwCwICBqwCAQEEAhYAMAsCAgatAgEBBAIMADALAgIGsAIBAQQCFgAwCwICBrICAQEEAgwAMAsCAgazAgEBBAIMADALAgIGtAIBAQQCDAAwCwICBrUCAQEEAgwAMAsCAga2AgEBBAIMADAMAgIGpQIBAQQDAgEBMAwCAgarAgEBBAMCAQEwDAICBq4CAQEEAwIBADAMAgIGrwIBAQQDAgEAMAwCAgaxAgEBBAMCAQAwEAICBqYCAQEEBwwFdGVzdDIwGwICBqcCAQEEEgwQMTAwMDAwMDQ3MjEwNjA4MjAbAgIGqQIBAQQSDBAxMDAwMDAwNDcyMTA2MDgyMB8CAgaoAgEBBBYWFDIwMTgtMTEtMTNUMTY6NDY6MzFaMB8CAgaqAgEBBBYWFDIwMTgtMTEtMTNUMTY6NDY6MzFaoIIOZTCCBXwwggRkoAMCAQICCA7rV4fnngmNMA0GCSqGSIb3DQEBBQUAMIGWMQswCQYDVQQGEwJVUzETMBEGA1UECgwKQXBwbGUgSW5jLjEsMCoGA1UECwwjQXBwbGUgV29ybGR3aWRlIERldmVsb3BlciBSZWxhdGlvbnMxRDBCBgNVBAMMO0FwcGxlIFdvcmxkd2lkZSBEZXZlbG9wZXIgUmVsYXRpb25zIENlcnRpZmljYXRpb24gQXV0aG9yaXR5MB4XDTE1MTExMzAyMTUwOVoXDTIzMDIwNzIxNDg0N1owgYkxNzA1BgNVBAMMLk1hYyBBcHAgU3RvcmUgYW5kIGlUdW5lcyBTdG9yZSBSZWNlaXB0IFNpZ25pbmcxLDAqBgNVBAsMI0FwcGxlIFdvcmxkd2lkZSBEZXZlbG9wZXIgUmVsYXRpb25zMRMwEQYDVQQKDApBcHBsZSBJbmMuMQswCQYDVQQGEwJVUzCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAKXPgf0looFb1oftI9ozHI7iI8ClxCbLPcaf7EoNVYb/pALXl8o5VG19f7JUGJ3ELFJxjmR7gs6JuknWCOW0iHHPP1tGLsbEHbgDqViiBD4heNXbt9COEo2DTFsqaDeTwvK9HsTSoQxKWFKrEuPt3R+YFZA1LcLMEsqNSIH3WHhUa+iMMTYfSgYMR1TzN5C4spKJfV+khUrhwJzguqS7gpdj9CuTwf0+b8rB9Typj1IawCUKdg7e/pn+/8Jr9VterHNRSQhWicxDkMyOgQLQoJe2XLGhaWmHkBBoJiY5uB0Qc7AKXcVz0N92O9gt2Yge4+wHz+KO0NP6JlWB7+IDSSMCAwEAAaOCAdcwggHTMD8GCCsGAQUFBwEBBDMwMTAvBggrBgEFBQcwAYYjaHR0cDovL29jc3AuYXBwbGUuY29tL29jc3AwMy13d2RyMDQwHQYDVR0OBBYEFJGknPzEdrefoIr0TfWPNl3tKwSFMAwGA1UdEwEB/wQCMAAwHwYDVR0jBBgwFoAUiCcXCam2GGCL7Ou69kdZxVJUo7cwggEeBgNVHSAEggEVMIIBETCCAQ0GCiqGSIb3Y2QFBgEwgf4wgcMGCCsGAQUFBwICMIG2DIGzUmVsaWFuY2Ugb24gdGhpcyBjZXJ0aWZpY2F0ZSBieSBhbnkgcGFydHkgYXNzdW1lcyBhY2NlcHRhbmNlIG9mIHRoZSB0aGVuIGFwcGxpY2FibGUgc3RhbmRhcmQgdGVybXMgYW5kIGNvbmRpdGlvbnMgb2YgdXNlLCBjZXJ0aWZpY2F0ZSBwb2xpY3kgYW5kIGNlcnRpZmljYXRpb24gcHJhY3RpY2Ugc3RhdGVtZW50cy4wNgYIKwYBBQUHAgEWKmh0dHA6Ly93d3cuYXBwbGUuY29tL2NlcnRpZmljYXRlYXV0aG9yaXR5LzAOBgNVHQ8BAf8EBAMCB4AwEAYKKoZIhvdjZAYLAQQCBQAwDQYJKoZIhvcNAQEFBQADggEBAA2mG9MuPeNbKwduQpZs0+iMQzCCX+Bc0Y2+vQ+9GvwlktuMhcOAWd/j4tcuBRSsDdu2uP78NS58y60Xa45/H+R3ubFnlbQTXqYZhnb4WiCV52OMD3P86O3GH66Z+GVIXKDgKDrAEDctuaAEOR9zucgF/fLefxoqKm4rAfygIFzZ630npjP49ZjgvkTbsUxn/G4KT8niBqjSl/OnjmtRolqEdWXRFgRi48Ff9Qipz2jZkgDJwYyz+I0AZLpYYMB8r491ymm5WyrWHWhumEL1TKc3GZvMOxx6GUPzo22/SGAGDDaSK+zeGLUR2i0j0I78oGmcFxuegHs5R0UwYS/HE6gwggQiMIIDCqADAgECAggB3rzEOW2gEDANBgkqhkiG9w0BAQUFADBiMQswCQYDVQQGEwJVUzETMBEGA1UEChMKQXBwbGUgSW5jLjEmMCQGA1UECxMdQXBwbGUgQ2VydGlmaWNhdGlvbiBBdXRob3JpdHkxFjAUBgNVBAMTDUFwcGxlIFJvb3QgQ0EwHhcNMTMwMjA3MjE0ODQ3WhcNMjMwMjA3MjE0ODQ3WjCBljELMAkGA1UEBhMCVVMxEzARBgNVBAoMCkFwcGxlIEluYy4xLDAqBgNVBAsMI0FwcGxlIFdvcmxkd2lkZSBEZXZlbG9wZXIgUmVsYXRpb25zMUQwQgYDVQQDDDtBcHBsZSBXb3JsZHdpZGUgRGV2ZWxvcGVyIFJlbGF0aW9ucyBDZXJ0aWZpY2F0aW9uIEF1dGhvcml0eTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAMo4VKbLVqrIJDlI6Yzu7F+4fyaRvDRTes58Y4Bhd2RepQcjtjn+UC0VVlhwLX7EbsFKhT4v8N6EGqFXya97GP9q+hUSSRUIGayq2yoy7ZZjaFIVPYyK7L9rGJXgA6wBfZcFZ84OhZU3au0Jtq5nzVFkn8Zc0bxXbmc1gHY2pIeBbjiP2CsVTnsl2Fq/ToPBjdKT1RpxtWCcnTNOVfkSWAyGuBYNweV3RY1QSLorLeSUheHoxJ3GaKWwo/xnfnC6AllLd0KRObn1zeFM78A7SIym5SFd/Wpqu6cWNWDS5q3zRinJ6MOL6XnAamFnFbLw/eVovGJfbs+Z3e8bY/6SZasCAwEAAaOBpjCBozAdBgNVHQ4EFgQUiCcXCam2GGCL7Ou69kdZxVJUo7cwDwYDVR0TAQH/BAUwAwEB/zAfBgNVHSMEGDAWgBQr0GlHlHYJ/vRrjS5ApvdHTX8IXjAuBgNVHR8EJzAlMCOgIaAfhh1odHRwOi8vY3JsLmFwcGxlLmNvbS9yb290LmNybDAOBgNVHQ8BAf8EBAMCAYYwEAYKKoZIhvdjZAYCAQQCBQAwDQYJKoZIhvcNAQEFBQADggEBAE/P71m+LPWybC+P7hOHMugFNahui33JaQy52Re8dyzUZ+L9mm06WVzfgwG9sq4qYXKxr83DRTCPo4MNzh1HtPGTiqN0m6TDmHKHOz6vRQuSVLkyu5AYU2sKThC22R1QbCGAColOV4xrWzw9pv3e9w0jHQtKJoc/upGSTKQZEhltV/V6WId7aIrkhoxK6+JJFKql3VUAqa67SzCu4aCxvCmA5gl35b40ogHKf9ziCuY7uLvsumKV8wVjQYLNDzsdTJWk26v5yZXpT+RN5yaZgem8+bQp0gF6ZuEujPYhisX4eOGBrr/TkJ2prfOv/TgalmcwHFGlXOxxioK0bA8MFR8wggS7MIIDo6ADAgECAgECMA0GCSqGSIb3DQEBBQUAMGIxCzAJBgNVBAYTAlVTMRMwEQYDVQQKEwpBcHBsZSBJbmMuMSYwJAYDVQQLEx1BcHBsZSBDZXJ0aWZpY2F0aW9uIEF1dGhvcml0eTEWMBQGA1UEAxMNQXBwbGUgUm9vdCBDQTAeFw0wNjA0MjUyMTQwMzZaFw0zNTAyMDkyMTQwMzZaMGIxCzAJBgNVBAYTAlVTMRMwEQYDVQQKEwpBcHBsZSBJbmMuMSYwJAYDVQQLEx1BcHBsZSBDZXJ0aWZpY2F0aW9uIEF1dGhvcml0eTEWMBQGA1UEAxMNQXBwbGUgUm9vdCBDQTCCASIwDQYJKoZIhvcNAQEBBQADggEPADCCAQoCggEBAOSRqQkfkdseR1DrBe1eeYQt6zaiV0xV7IsZid75S2z1B6siMALoGD74UAnTf0GomPnRymacJGsR0KO75Bsqwx+VnnoMpEeLW9QWNzPLxA9NzhRp0ckZcvVdDtV/X5vyJQO6VY9NXQ3xZDUjFUsVWR2zlPf2nJ7PULrBWFBnjwi0IPfLrCwgb3C2PwEwjLdDzw+dPfMrSSgayP7OtbkO2V4c1ss9tTqt9A8OAJILsSEWLnTVPA3bYharo3GSR1NVwa8vQbP4++NwzeajTEV+H0xrUJZBicR0YgsQg0GHM4qBsTBY7FoEMoxos48d3mVz/2deZbxJ2HafMxRloXeUyS0CAwEAAaOCAXowggF2MA4GA1UdDwEB/wQEAwIBBjAPBgNVHRMBAf8EBTADAQH/MB0GA1UdDgQWBBQr0GlHlHYJ/vRrjS5ApvdHTX8IXjAfBgNVHSMEGDAWgBQr0GlHlHYJ/vRrjS5ApvdHTX8IXjCCAREGA1UdIASCAQgwggEEMIIBAAYJKoZIhvdjZAUBMIHyMCoGCCsGAQUFBwIBFh5odHRwczovL3d3dy5hcHBsZS5jb20vYXBwbGVjYS8wgcMGCCsGAQUFBwICMIG2GoGzUmVsaWFuY2Ugb24gdGhpcyBjZXJ0aWZpY2F0ZSBieSBhbnkgcGFydHkgYXNzdW1lcyBhY2NlcHRhbmNlIG9mIHRoZSB0aGVuIGFwcGxpY2FibGUgc3RhbmRhcmQgdGVybXMgYW5kIGNvbmRpdGlvbnMgb2YgdXNlLCBjZXJ0aWZpY2F0ZSBwb2xpY3kgYW5kIGNlcnRpZmljYXRpb24gcHJhY3RpY2Ugc3RhdGVtZW50cy4wDQYJKoZIhvcNAQEFBQADggEBAFw2mUwteLftjJvc83eb8nbSdzBPwR+Fg4UbmT1HN/Kpm0COLNSxkBLYvvRzm+7SZA/LeU802KI++Xj/a8gH7H05g4tTINM4xLG/mk8Ka/8r/FmnBQl8F0BWER5007eLIztHo9VvJOLr0bdw3w9F4SfK8W147ee1Fxeo3H4iNcol1dkP1mvUoiQjEfehrI9zgWDGG1sJL5Ky+ERI8GA4nhX1PSZnIIozavcNgs/e66Mv+VNqW2TAYzN39zoHLFbr2g8hDtq6cxlPtdk2f8GHVdmnmbkyQvvY1XGefqFStxu9k0IkEirHDx22TZxeY8hLgBdQqorV2uT80AkHN7B1dSExggHLMIIBxwIBATCBozCBljELMAkGA1UEBhMCVVMxEzARBgNVBAoMCkFwcGxlIEluYy4xLDAqBgNVBAsMI0FwcGxlIFdvcmxkd2lkZSBEZXZlbG9wZXIgUmVsYXRpb25zMUQwQgYDVQQDDDtBcHBsZSBXb3JsZHdpZGUgRGV2ZWxvcGVyIFJlbGF0aW9ucyBDZXJ0aWZpY2F0aW9uIEF1dGhvcml0eQIIDutXh+eeCY0wCQYFKw4DAhoFADANBgkqhkiG9w0BAQEFAASCAQCJ9ctD+7Yi9JWvl6G+1HOcDO++mhY6rc6japAgogVF4xmIdh275IKRwZKpQbhoJmxXwElbMjkIsXks/48/EzuaHDQBNIVowq8qQaSUb3msvfAZfi7RGnhaJGzkXf7azr9NLMxX29R2jTiw2oaz2ri49piggmrGfXsLjWs9zTHWHHNRN1fLTPtcWb95JbQNAiQqlecG5a95/+KZ7+joh8fQwbthe8oWs5Tla0DDwrEoIbc5yjFT18Dln5bndTvWQJZcsbI4xa7BAEhjg/nfwPhaL17tHZeW8mOcCtG9UcuAgXXC6usVAOSocenhmKUR8W+D6F/jhBn0k9ahApPDmpZh"

In [68]:
receipt_data = base64.b64decode(textBytes)
# print(base64.b64decode(sample.encode("shift-jis")).decode(errors="ignore"))
print(receipt_data)

b'0\x82&V\x06\t*\x86H\x86\xf7\r\x01\x07\x02\xa0\x82&G0\x82&C\x02\x01\x011\x0b0\t\x06\x05+\x0e\x03\x02\x1a\x05\x000\x82\x15\x94\x06\t*\x86H\x86\xf7\r\x01\x07\x01\xa0\x82\x15\x85\x04\x82\x15\x811\x82\x15}0\n\x02\x01\x08\x02\x01\x01\x04\x02\x16\x000\n\x02\x01\x14\x02\x01\x01\x04\x02\x0c\x000\x0b\x02\x01\x01\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0b\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0f\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x10\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x19\x02\x01\x01\x04\x03\x02\x01\x030\x0c\x02\x01\x03\x02\x01\x01\x04\x04\x0c\x02180\x0c\x02\x01\n\x02\x01\x01\x04\x04\x16\x024+0\x0c\x02\x01\x0e\x02\x01\x01\x04\x04\x02\x02\x00\x9d0\r\x02\x01\r\x02\x01\x01\x04\x05\x02\x03\x02sY0\r\x02\x01\x13\x02\x01\x01\x04\x05\x0c\x031.00\x0e\x02\x01\t\x02\x01\x01\x04\x06\x02\x04P3010\x18\x02\x01\x04\x02\x01\x02\x04\x10\x19+\x92\x0f3\x92\xb0)\x12\xa7\xd6\xd4\x1d\xbd\xc7\xe80\x1b\x02\x01\x00\x02\x01\x01\x04\x13\x0c\x11ProductionSandbox0\x1c\x02\x01\x05\x02\x01\x0

In [82]:
from asn1crypto import cms

In [84]:
from pyasn1.codec.der import decoder
from pyasn1.codec.der import encoder

from pyasn1_modules import pem
from pyasn1_modules import rfc2315

In [85]:
contentInfo, rest = decoder.decode(receipt_data, asn1Spec=rfc2315.ContentInfo())

In [93]:
contentType = contentInfo.getComponentByName('contentType')
# contentInfo.getComponentByName('contentType')

In [92]:
contentInfoMap = {
    (1, 2, 840, 113549, 1, 7, 1): rfc2315.Data(),
    (1, 2, 840, 113549, 1, 7, 2): rfc2315.SignedData(),
    (1, 2, 840, 113549, 1, 7, 3): rfc2315.EnvelopedData(),
    (1, 2, 840, 113549, 1, 7, 4): rfc2315.SignedAndEnvelopedData(),
    (1, 2, 840, 113549, 1, 7, 5): rfc2315.DigestedData(),
    (1, 2, 840, 113549, 1, 7, 6): rfc2315.EncryptedData()
}

In [94]:
content, _ = decoder.decode(
    contentInfo.getComponentByName('content'),
    asn1Spec=contentInfoMap[contentType]
)


In [95]:
content

<SignedData value object, tagSet=<TagSet object, tags 0:32:16>, subtypeSpec=<ConstraintsIntersection object>, componentType=<NamedTypes object, types <NamedType object, type version=<Version schema object, tagSet <TagSet object, tags 0:0:2>>>, <OptionalNamedType object, type digestAlgorithms=<DigestAlgorithmIdentifiers schema object, tagSet=<TagSet object, tags 0:32:17>, subtypeSpec=<ConstraintsIntersection object>, componentType=<DigestAlgorithmIdentifier schema object, tagSet=<TagSet object, tags 0:32:16>, subtypeSpec=<ConstraintsIntersection object>, componentType=<NamedTypes object, types <NamedType object, type algorithm=<ObjectIdentifier schema object, tagSet <TagSet object, tags 0:0:6>>>, <OptionalNamedType object, type parameters=<Any schema object, encoding iso-8859-1>>>, sizeSpec=<ConstraintsIntersection object>>, sizeSpec=<ConstraintsIntersection object>>>, <NamedType object, type contentInfo=<ContentInfo schema object, tagSet=<TagSet object, tags 0:32:16>, subtypeSpec=<Cons

In [96]:
print(content.prettyPrint())

SignedData:
 version=1
 digestAlgorithms=DigestAlgorithmIdentifiers:
  DigestAlgorithmIdentifier:
   algorithm=1.3.14.3.2.26
   parameters=0x0500

 contentInfo=ContentInfo:
  contentType=1.2.840.113549.1.7.1
  content=0x048215813182157d300a02010802010104021600300a02011402010104020c00300b0201010201010403020100300b02010b0201010403020100300b02010f0201010403020100300b0201100201010403020100300b0201190201010403020103300c02010302010104040c023138300c02010a02010104041602342b300c02010e02010104040202009d300d02010d02010104050203027359300d02011302010104050c03312e30300e020109020101040602045033303130180201040201020410192b920f3392b02912a7d6d41dbdc7e8301b02010002010104130c1150726f64756374696f6e53616e64626f78301c020105020101041408469b0c0f9101e9b00009bf4bbd60650cfb7ea4301e02010202010104160c14636f6d2e68616e616269696e632e68616e616269301e02010c02010104161614323032332d30392d31335431353a34303a35385a301e02011202010104161614323031332d30382d30315430373a30303a30305a304f0201060201010447bd64994f2d4b33b8d4dd85b2ff45

In [99]:
print(_.prettyPrint())

In [87]:
rest

b''

In [83]:
if cms.detect(receipt_data):
        _, _, der_bytes = cms.unarmor(receipt_data)

AttributeError: module 'asn1crypto.cms' has no attribute 'detect'

In [ ]:
print(receipt_data.decode("shift-jis"))

In [32]:
from base64 import b64encode, b64decode


s = '本日は晴天なり'
encoded = b64encode(s.encode())
decoded = b64decode(encoded)

print(encoded)
print(decoded)
# b'\xe6\x9c\xac\xe6\x97\xa5\xe3\x81\xaf\xe6\x99\xb4\xe5\xa4\xa9\xe3\x81\xaa\xe3\x82\x8a'

print(decoded.decode())
# 本日は晴天なり

b'5pys5pel44Gv5pm05aSp44Gq44KK'
b'\xe6\x9c\xac\xe6\x97\xa5\xe3\x81\xaf\xe6\x99\xb4\xe5\xa4\xa9\xe3\x81\xaa\xe3\x82\x8a'
本日は晴天なり


In [26]:
textBytes.decode()

'MIImVgYJKoZIhvcNAQcCoIImRzCCJkMCAQExCzAJBgUrDgMCGgUAMIIVlAYJKoZIhvcNAQcBoIIVhQSCFYExghV9MAoCAQgCAQEEAhYAMAoCARQCAQEEAgwAMAsCAQECAQEEAwIBADALAgELAgEBBAMCAQAwCwIBDwIBAQQDAgEAMAsCARACAQEEAwIBADALAgEZAgEBBAMCAQMwDAIBAwIBAQQEDAIxODAMAgEKAgEBBAQWAjQrMAwCAQ4CAQEEBAICAJ0wDQIBDQIBAQQFAgMCc1kwDQIBEwIBAQQFDAMxLjAwDgIBCQIBAQQGAgRQMzAxMBgCAQQCAQIEEBkrkg8zkrApEqfW1B29x+gwGwIBAAIBAQQTDBFQcm9kdWN0aW9uU2FuZGJveDAcAgEFAgEBBBQIRpsMD5EB6bAACb9LvWBlDPt+pDAeAgECAgEBBBYMFGNvbS5oYW5hYmlpbmMuaGFuYWJpMB4CAQwCAQEEFhYUMjAyMy0wOS0xM1QxNTo0MDo1OFowHgIBEgIBAQQWFhQyMDEzLTA4LTAxVDA3OjAwOjAwWjBPAgEGAgEBBEe9ZJlPLUszuNTdhbL/RcAHDYMan8cuZDr2y8eZJKewKTWktJh+7L4+Fhkb00XmEVL6n+SzbusLNZDSoDUr6fhmqjsmWxmp+DBQAgEHAgEBBEhWqXEIrdBmMRxdH7yrRwgBBFpTkgnJBMsPVJDFIAozMmWJGWYJmZCh5AOdAd3Oi/gty+IQ1YGlEIGJ12HvaEfKF9McuNdQuQcwggGbAgERAgEBBIIBkTGCAY0wCwICBq0CAQEEAgwAMAsCAgawAgEBBAIWADALAgIGsgIBAQQCDAAwCwICBrMCAQEEAgwAMAsCAga0AgEBBAIMADALAgIGtQIBAQQCDAAwCwICBrYCAQEEAgwAMAwCAgalAgEBBAMCAQEwDAICBqsCAQEEAwIBAzAMAgIGrgIBAQQDAgEAMAwCAgaxAgEBBAM

In [77]:
import base64
import hashlib
from Crypto import Random
from Crypto.Cipher import AES
from Crypto.Util import Padding

In [79]:
key = "312f4d460b7e456d92b51d51ccb15fe5"
key = (hashlib.md5(key.encode('utf-8')).hexdigest()).encode('utf-8')

In [80]:
def decrypt(enc):
    enc = base64.b64decode(enc)
    iv = enc[:AES.block_size]
    cipher = AES.new(key, AES.MODE_CBC, iv)
    data = Padding.unpad(cipher.decrypt(enc[AES.block_size:]), AES.block_size, 'pkcs7')
    return data.decode('utf-8')

In [ ]:
print(decrypt(receipt_data))

In [16]:
decoder = asn1.Decoder()

In [69]:
decoder.start(receipt_data)
tag, value = decoder.read()

In [70]:
print(tag)

Tag(nr=16, typ=32, cls=0)


In [71]:
print(value)

b'\x06\t*\x86H\x86\xf7\r\x01\x07\x02\xa0\x82&G0\x82&C\x02\x01\x011\x0b0\t\x06\x05+\x0e\x03\x02\x1a\x05\x000\x82\x15\x94\x06\t*\x86H\x86\xf7\r\x01\x07\x01\xa0\x82\x15\x85\x04\x82\x15\x811\x82\x15}0\n\x02\x01\x08\x02\x01\x01\x04\x02\x16\x000\n\x02\x01\x14\x02\x01\x01\x04\x02\x0c\x000\x0b\x02\x01\x01\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0b\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0f\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x10\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x19\x02\x01\x01\x04\x03\x02\x01\x030\x0c\x02\x01\x03\x02\x01\x01\x04\x04\x0c\x02180\x0c\x02\x01\n\x02\x01\x01\x04\x04\x16\x024+0\x0c\x02\x01\x0e\x02\x01\x01\x04\x04\x02\x02\x00\x9d0\r\x02\x01\r\x02\x01\x01\x04\x05\x02\x03\x02sY0\r\x02\x01\x13\x02\x01\x01\x04\x05\x0c\x031.00\x0e\x02\x01\t\x02\x01\x01\x04\x06\x02\x04P3010\x18\x02\x01\x04\x02\x01\x02\x04\x10\x19+\x92\x0f3\x92\xb0)\x12\xa7\xd6\xd4\x1d\xbd\xc7\xe80\x1b\x02\x01\x00\x02\x01\x01\x04\x13\x0c\x11ProductionSandbox0\x1c\x02\x01\x05\x02\x01\x01\x04\x

In [74]:
value.decode(errors="ignore")

'\x06\t*H\r\x01\x07\x02&G0&C\x02\x01\x011\x0b0\t\x06\x05+\x0e\x03\x02\x1a\x05\x000\x15\x06\t*H\r\x01\x07\x01\x15\x04\x151\x15}0\n\x02\x01\x08\x02\x01\x01\x04\x02\x16\x000\n\x02\x01\x14\x02\x01\x01\x04\x02\x0c\x000\x0b\x02\x01\x01\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0b\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x0f\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x10\x02\x01\x01\x04\x03\x02\x01\x000\x0b\x02\x01\x19\x02\x01\x01\x04\x03\x02\x01\x030\x0c\x02\x01\x03\x02\x01\x01\x04\x04\x0c\x02180\x0c\x02\x01\n\x02\x01\x01\x04\x04\x16\x024+0\x0c\x02\x01\x0e\x02\x01\x01\x04\x04\x02\x02\x000\r\x02\x01\r\x02\x01\x01\x04\x05\x02\x03\x02sY0\r\x02\x01\x13\x02\x01\x01\x04\x05\x0c\x031.00\x0e\x02\x01\t\x02\x01\x01\x04\x06\x02\x04P3010\x18\x02\x01\x04\x02\x01\x02\x04\x10\x19+\x0f3)\x12\x1d0\x1b\x02\x01\x00\x02\x01\x01\x04\x13\x0c\x11ProductionSandbox0\x1c\x02\x01\x05\x02\x01\x01\x04\x14\x08F\x0c\x0f\x01\x00\tK`e\x0c~0\x1e\x02\x01\x02\x02\x01\x01\x04\x16\x0c\x14com.hanabiinc.hanabi0\x1e\x02\x